In [1]:
import torch 
from torch import nn
from torch import Tensor

device = "cuda" if torch.cuda.is_available() else "cpu"

print(device)



cuda


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
class My_CBOW(nn.Module):
    def __init__(self, vob_size:int, embed_dim:int, window_size:int):
        super(My_CBOW,self).__init__()
        self.inEmbedding = nn.Embedding(vob_size,embed_dim)
      
        size = embed_dim * window_size * 2;

#        print(f"size =={size}")
        # self.linear1 = nn.Linear(size, embed_dim)

        self.outEmbedding = nn.Embedding(vob_size,embed_dim)

    #    self.h = None
        
    
    def makeH(self,input):
        W = self.inEmbedding(input)
        n = W.shape[-2]


  #      print(f"W.shape = {W.shape}")
        h = torch.sum(W,dim=-2) / n
    #     x = W.view(input.shape[0],-1)


    #  #   print(f"x.shape = {x.shape}")
    #     h = self.linear1(x)
    #     h = nn.functional.relu(h)

        return h

    def forwardWithH(self,h,out):
        W = self.outEmbedding(out)
        score = torch.sum(h * W,dim = -1)
        return score



# ## 模型测试
# cbow = My_CBOW(10,3,3)
# cbow.to(device=device)
# input = torch.Tensor([[1,2,5],[1,2,3],[6,2,3],[6,2,3]]).to(dtype=torch.int32,device = device)

# ##正样本
# h = cbow.makeH(input)

# print(f"shape{h.shape}")
# pout = Tensor([1,3,2,5]).to(device=device,dtype=int)
# plable = torch.ones(pout.shape).to(device = device)
# pscore = cbow.forwardWithH(h,pout).to(device = device)


# ##负样本
# nout = Tensor([1,0,1,7]).to(device=device,dtype=int)
# nlable = torch.zeros_like(nout).to(device = device)
# nsocre = cbow.forwardWithH(h,nout).to(device = device)


# score = torch.cat((pscore,nsocre))
# lables = torch.cat((plable,nlable)).to(device = device)
# print(score)



# loss_fn = torch.nn.BCEWithLogitsLoss()
# loss = loss_fn(score,lables)

# print(loss)

# loss.backward()

# input = Tensor([[1,2,5],[1,2,3],[6,2,3]],dtype=torch.int)
# print(input)

# cbow.makeH(input)



In [5]:
import sys

sys.path.append("/content/drive/MyDrive/ml/nlp/")

import numpy
import pickle


from common.util import create_contexts_target
from dataset import ptb


window_size = 5
hidden_size =  100

batch_size = 100
max_epoch = 10


corpus, word_to_id, id_to_word = ptb.load_data("train")
vocab_size = len(word_to_id)

context , target = create_contexts_target(corpus=corpus,window_size=window_size)

print(context.shape)
print(target)



(929579, 10)
[   5    6    7 ...  552  917 3196]


In [6]:
import collections
import numpy as np
GPU = device == "cuda"
class UnigramSampler:
    def __init__(self, corpus, power, sample_size):
        self.sample_size = sample_size
        self.vocab_size = None
        self.word_p = None

        counts = collections.Counter()
        for word_id in corpus:
            counts[word_id] += 1

        vocab_size = len(counts)
        self.vocab_size = vocab_size

        self.word_p = np.zeros(vocab_size)
        for i in range(vocab_size):
            self.word_p[i] = counts[i]

        self.word_p = np.power(self.word_p, power)
        self.word_p /= np.sum(self.word_p)
    def get_negative_sample(self, target):
        batch_size = target.shape[0]

        if not GPU:
            negative_sample = np.zeros((batch_size, self.sample_size), dtype=np.int32)

            for i in range(batch_size):
                p = self.word_p.copy()
                target_idx = target[i]
                p[target_idx] = 0
                p /= p.sum()
                negative_sample[i, :] = np.random.choice(self.vocab_size, size=self.sample_size, replace=False, p=p)
        else:
            # 在用GPU(cupy）计算时，优先速度
            # 有时目标词存在于负例中
            negative_sample = np.random.choice(self.vocab_size, size=(batch_size, self.sample_size),
                                               replace=True, p=self.word_p)

        return negative_sample

In [31]:
def train(model:My_CBOW, loss_fn , optimizer:torch.optim.Optimizer ,context : Tensor, target: Tensor, negativeTarget: Tensor):

  
    # model.train()
    h = model.makeH(context)

    p_lable = torch.ones_like(target)
    

    p_score = model.forwardWithH(h,target)

    n_lable = torch.zeros_like(negativeTarget)
    n_score = model.forwardWithH(h,negativeTarget)

    score = torch.cat((p_score,n_score)).to(dtype=torch.float32)
    lables = torch.cat((p_lable,n_lable)).to(dtype=torch.float32)
    loss = loss_fn(score,lables)
    optimizer.zero_grad()
    loss.backward()

    #optimizer.zero_grad()
    optimizer.step()
    return loss

In [14]:
train_context = torch.tensor(context).to(device=device)
train_target = torch.tensor(target).to(device = device)

# print(train_context.shape)
# print(train_target.shape)

# model = My_CBOW(vocab_size,hidden_size)
# optimizer = torch.optim.Adam(model.parameters())
# loss_fn = torch.nn.BCEWithLogitsLoss()




# ## 取十条负样本
# #sampler = torch.utils.data.Sampler(data_source)

# smapler = UnigramSampler(corpus,0.75,1)


# print(target.shape)


# batch_context = train_context[:10]
# batch_target = train_target[:10]



# negative = smapler.get_negative_sample(batch_target)
# print(negative.shape)

# negative = torch.from_numpy(negative).squeeze(dim=1)
# ##model.train()

# print(f"batch_target_shape{batch_target.shape}")
# train(model,loss_fn,optimizer,batch_context,batch_target,negative)





In [24]:


model = My_CBOW(vocab_size,hidden_size , window_size=window_size)
model = model.to(device=device)

loss_fn = torch.nn.BCEWithLogitsLoss()




                

In [32]:




 
smapler = UnigramSampler(corpus,0.75,5)


loss = 0.0
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)

model.train()

My_CBOW(
  (inEmbedding): Embedding(10000, 100)
  (outEmbedding): Embedding(10000, 100)
)

In [33]:
batch_size = 500
loop = train_target.shape[0] // batch_size


print(f"model weight : {model.inEmbedding.weight}")



for epoch in range(10):
    loss = 0.0
    for i in range(loop):
        start = i * batch_size
        end = (i + 1) * batch_size

        b_context = train_context[start : end]
        b_target =  train_target[start : end]

        negative = smapler.get_negative_sample(b_target)

        negatives = torch.from_numpy(negative.T).to(device= device)

        for j in range(5):
          negative = negatives[j]
          loss += train(model,loss_fn,optimizer,b_context,b_target,negative)


        # negative = smapler.get_negative_sample(b_target)
        # negative = torch.from_numpy(negative).squeeze(dim=1).to(device= device)





      
        # loss += train(model,loss_fn,optimizer,b_context,b_target,negative)

    print(f"{epoch} / {max_epoch} : losss is {loss/batch_size}")

print(f"model weight : {model.inEmbedding.weight}")

model weight : Parameter containing:
tensor([[ 0.7684,  0.4216,  0.0579,  ...,  0.7694,  1.4998,  0.4847],
        [-0.8013,  0.1884, -0.5305,  ...,  0.5169,  1.2377, -0.3272],
        [ 0.9748,  0.0078,  1.1083,  ..., -0.9189,  0.3697, -1.7594],
        ...,
        [-0.2633,  1.3464, -0.3807,  ...,  1.7667,  0.1280, -0.5376],
        [ 3.3827,  1.3655, -0.1167,  ..., -0.2156, -1.5520, -0.0815],
        [-0.3408, -0.9149,  1.3900,  ...,  0.6726, -0.0064,  0.1280]],
       device='cuda:0', requires_grad=True)
0 / 10 : losss is 12.01616382598877
1 / 10 : losss is 8.862168312072754
2 / 10 : losss is 7.893669128417969
3 / 10 : losss is 7.434049129486084
4 / 10 : losss is 7.17485237121582


KeyboardInterrupt: ignored

## 保存模型


In [22]:
torch.save(model.state_dict(),"/content/drive/MyDrive/ml/nlp/cbow-ptb.pth")

## 加载模型


In [20]:


model.load_state_dict(torch.load("/content/drive/MyDrive/ml/nlp/cbow-ptb.pth"))

<All keys matched successfully>

In [34]:
from common.util import most_similar


## CBOW 模型的评价

print(model.inEmbedding.weight.shape)


word_vecs = model.inEmbedding.weight.detach().to(device = "cpu").numpy()

querys = ['you','year','car','toyota']

for query in querys:
  most_similar(query,word_to_id,id_to_word,word_vecs)




torch.Size([10000, 100])

[query] you
 i: 0.6182621717453003
 they: 0.607410728931427
 we: 0.5694349408149719
 nervously: 0.3941842317581177
 mom: 0.37667644023895264

[query] year
 week: 0.7344915270805359
 month: 0.7199020385742188
 summer: 0.6118643879890442
 spring: 0.5503232479095459
 decade: 0.48749762773513794

[query] car
 integrated: 0.3963068127632141
 cars: 0.3956888020038605
 dominated: 0.393172949552536
 chevy: 0.38974159955978394
 bid: 0.3842407166957855

[query] toyota
 cars: 0.4866105616092682
 goods: 0.42563217878341675
 chrysler: 0.4079570472240448
 beers: 0.39074578881263733
 afloat: 0.3547462821006775
